# Odometry and Avoidance

## Imports

In [111]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

## Global variable

In [112]:
##### ODOMETRY

## DO NOT CHANGE
CALIBRATION_TIME_ANGLE = 4.8
CALIBRATION_ANGLE = 180
CALIBRATION_TIME = 1.04
DISTANCE_CALLIBRATION = 3

FOURCHETTE = 5 # to uniform with the code
COTE_CARRE_MAP = 3



##### AVOIDANCE
## DO NOT CHANGE
SENSOR_0 = 0
SENSOR_1 = 1
SENSOR_2 = 2
SENSOR_3 = 3
SENSOR_4 = 4
SENSOR_5 = 5
SENSOR_6 = 6
ALL_FRONT_SENSOR = 5
PARTIALY_ALL_FRONT_SENSOR = 4
DIST_WHEELS = 9.4


PROXIMITY_THRESHOLD = 3800 #around 2cm
TIME_SLEEP_OBSTACLE = 0.1 #in seconds
DISTANCE_AVOIDANCE = 0 # to move away from the obstacle


#BOTH
MOTOR_SPEED = 80

In [113]:
def robot_turn(gamma, direction,alpha): #return alpha, direction, obstacle_seen (bool: 0->free)
    gamma = abs(gamma)
    time_turn = CALIBRATION_TIME_ANGLE*gamma/CALIBRATION_ANGLE
    obstacle_seen = 0
    
    if direction == "right":
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", int(int(2**16)-MOTOR_SPEED))
        t0 =time.time()
        while time.time()-t0 < time_turn :
            if (check_avoidance()) :
                obstacle_seen = 1
                avoid_obstacle()
                print('coucouR')
                return alpha, direction, obstacle_seen
        
        alpha = angle_between_pi(alpha-math.radians(gamma))
        
    if direction == "left":
        robot.set_var("motor.left.target", int(int(2**16)-MOTOR_SPEED))
        robot.set_var("motor.right.target", MOTOR_SPEED)
        t0 =time.time()
        while time.time()-t0 < time_turn :
            if (check_avoidance()) :
                obstacle_seen = 1
                avoid_obstacle()
                print('coucouL')
                return alpha, direction, obstacle_seen
        
        alpha = angle_between_pi(alpha+math.radians(gamma))
    
    direction = "forward"
    return alpha, direction, obstacle_seen

In [114]:
def angle_between_pi(angle): #return angle in [-pi,pi]
    if angle > math.pi :
        angle = angle - 2*math.pi
        return angle
    elif angle < -math.pi:
        angle = angle + 2*math.pi
        return angle
    else :
        return angle

In [115]:
def robot_move_forward(direction, distance,xr,xt,yr,yt): # distance is in cm , returns new xr, yr, obstacle_seen (bool:0->free)
    
    obstacle_seen = 0
    
    if direction == "forward" :
        time_forward = CALIBRATION_TIME*distance/DISTANCE_CALLIBRATION
        
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", MOTOR_SPEED)
        
        t0 =time.time()
        while time.time()-t0 < time_forward :
            if (check_avoidance()) :
                obstacle_seen = 1
                avoid_obstacle()
                print('coucouF')
                return xr, yr, obstacle_seen

        xr=xt
        yr=yt
        
    return xr, yr, obstacle_seen

In [116]:
def robot_stop() :
    robot.set_var("motor.left.target", 0)
    robot.set_var("motor.right.target", 0)
    return

In [117]:
def robot_evaluation (xr,xt,yr,yt, alpha): #return gamma in degrees, direction, distance in cm
    
    alpha = math.degrees(alpha)
    xr=xr*COTE_CARRE_MAP
    xt=xt*COTE_CARRE_MAP
    yr=yr*COTE_CARRE_MAP
    yt=yt*COTE_CARRE_MAP
    
    if yt>yr and xt<xr :
        gamma = math.degrees(math.atan2(yt-yr,xr-xt))

        if alpha<=0 :
            gamma = gamma + 180+alpha

            if gamma < 180 :
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn left
                gamma = 360-gamma
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = 180 - gamma - alpha

            if gamma<0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)


    if yt<yr and xt<xr :  
        gamma = math.degrees(math.atan2(yr-yt,xr-xt))

        if alpha>=0 :
            gamma = gamma +180-alpha

            if gamma < 180 :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn right
                gamma = 360-gamma
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = 180 - gamma + alpha

            if gamma>0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)


    if yt<yr and xt>xr :  
        gamma = math.degrees(math.atan2(yr-yt,xt-xr))

        if alpha>=0 :
            gamma = gamma + alpha

            if gamma > 180 :
                #robot turn left
                gamma = 360-gamma
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = -alpha - gamma

            if gamma<0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        

    if yt>yr and xt>xr :
        gamma = math.degrees(math.atan2(yt-yr,xt-xr))

        if alpha<=0 :
            gamma = gamma - alpha

            if gamma > 180 :
                #robot turn right
                gamma = 360-gamma
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = alpha - gamma

            if gamma>0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
    
    
    if yt==yr and xt>xr :
        gamma = 0
        distance = xt-xr
        
        if abs(alpha)< FOURCHETTE :
            #robot move forward
            return abs(gamma), "forward", distance
        elif alpha < -(FOURCHETTE/2) :
            #robot turn left
            return abs(gamma), "left", distance
        else :
            #robot turn right
            return abs(gamma), "right", distance
            
    if yt==yr and xt<xr :
        gamma = 180
        distance = xr-xt

        if alpha < 0 :
            gamma = gamma + alpha
            #robot turn right
            return abs(gamma), "right", distance
        else:
            gamma = gamma - alpha
            #robot turn left
            return abs(gamma), "left", distance
            
    
    if yt<yr and xt==xr :
        gamma = 90                           #=-90
        gamma = gamma+alpha
        distance = yr-yt

        if gamma < 0:
            #robot turn left
            return abs(gamma), "left", distance
        elif gamma > 180 :
            #robot turn left
            gamma= 360-gamma
            return abs(gamma), "left", distance
        else:
            #robot turn right
            return abs(gamma), "right", distance
            
            
    if yt>yr and xt==xr :
        gamma = 90
        gamma = gamma-alpha
        distance = yt-yr

        if gamma < 0 :
            #robot turn right
            return abs(gamma), "right", distance
        elif gamma > 180 :
            #robot turn right
            gamma= 360-gamma
            return abs(gamma), "right", distance
        else:
            #robot turn left
            return abs(gamma), "left", distance

In [118]:
def target_not_reach(xr,xt,yr,yt): # return bool yes/no
    if yt==yr and xt==xr :
        return False
    else:
        return True

In [119]:
def obstacle_avoidance(obstacle_direction,sensor_measurment):
    if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
        if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
            robot.set_var("motor.left.target", MOTOR_SPEED)
            robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
        else:
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
    else:
        if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
            robot.set_var("motor.left.target", MOTOR_SPEED)
            robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
        elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
        elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
    return

In [120]:
def check_avoidance() :
    sensor_measurment = robot["prox.horizontal"]
    if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
        return 0
    else:
        return 1

In [124]:
def avoid_obstacle () :
    sensor_measurment = robot["prox.horizontal"]
    obstacle_direction = np.nonzero(sensor_measurment)[0]
    while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):

        obstacle_avoidance(obstacle_direction,sensor_measurment)

        time.sleep(TIME_SLEEP_OBSTACLE)

        sensor_measurment = robot["prox.horizontal"]
        obstacle_direction = np.nonzero(sensor_measurment)[0]
    
    robot_move_forward("forward", DISTANCE_AVOIDANCE,0,0,0,0)
    robot_stop()
    
    return

In [126]:
# We got :
target_list = [[2,2],[3,2],[4,3],[5,4],[5,5]] #for 5 target
#target_list2 = [[2,3,4,5,5][2,2,3,4,5]]

robot_pos=[1,2]
alpha = 0

#img = get_image()
#(robot_pos, start, goal) = get_data(img)
#alpha = robot_pos[2] # in radian

for target in target_list :
    
    #print('target: ', target)

    while (target_not_reach(robot_pos[0],target[0],robot_pos[1],target[1])):
        gamma, direction, distance = robot_evaluation (robot_pos[0],target[0],robot_pos[1],target[1], alpha)

        #print('gamma: ',gamma,'direction: ',direction,'distance: ',distance)

        alpha, direction, obstacle_seen = robot_turn(gamma, direction, alpha)

        #print('alpha: ',math.degrees(alpha),'direction: ',direction)
        while(obstacle_seen):
            #img = get_image()
            #(robot_pos, start, goal) = get_data(img)
            #alpha = robot_pos[2] # in radian
            
            print(obstacle_seen)
            obstacle_seen =0
            
            gamma, direction, distance = robot_evaluation (robot_pos[0],target[0],robot_pos[1],target[1], alpha)
            alpha, direction, obstacle_seen = robot_turn(gamma, direction, alpha)

        robot_pos[0], robot_pos[1], obstacle_seen = robot_move_forward(direction, distance,robot_pos[0],target[0],robot_pos[1],target[1])
        print(obstacle_seen)
        #if(obstacle_seen):
            #img = get_image()
            #(robot_pos, start, goal) = get_data(img)
            #alpha = robot_pos[2] # in radian

        #print('robot: ',robot_pos)

        robot_stop()
        
    

coucouF
1
0
0
0
0
0


In [123]:
robot_stop()